In [0]:
file_path = ("/Workspace/Users/deepakpvinodsharma@gmail.com/THE_LEARNING_BRICK/Databricks-begining-to-end/Streaming project/data for streamig.json")

In [0]:
from pyspark.sql.types import StructType, StringType, IntegerType, TimestampType
schema = StructType()\
    .add("id", StringType())\
    .add("Timestamp", TimestampType())\
    .add("method", StringType())\
    .add("endpoint", StringType())\
    .add("status", IntegerType())

In [0]:
# now we read the log
log_stream = spark.readStream \
    .schema(schema)\
    .option("maxFilesPerTrigger", 1)\
    .json(file_path)

In [0]:
# count request per endpoint
endpoint_counts = log_stream.groupBy("endpoint").count()

In [0]:
query = endpoint_counts.writeStream \
    .outputMode("complete")\
    .format("console")\
    .option("truncate", false)\
    .start()

In [0]:
top_ips = log_stream.groupBy("ip").count().orderBy("count", ascending=False)

query2 = top_ips.writeStream.outputMode("complete").format("console").start()

In [0]:
from pyspark.sql.functions import col

errors = log_stream.filter(col("status") >= 400)

error_counts = errors.groupBy("status").count()

query3 = error_counts.writeStream.outputMode("complete").format("console").start()

In [0]:
log_stream.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "/tmp/logs/checkpoints/") \
    .start("/mnt/delta/logs/")
